# Import

In [63]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
from torch import nn

# Hyperparameters

In [2]:
batch_size = 64

# Prepare dataset

In [64]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

# Network Utils

In [51]:
class Linear():
    def __init__(self,input_dim: int, output_dim: int, bias: np.ndarray = None):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.weights = np.random.rand(self.output_dim, self.input_dim).astype(np.float32)
        self.weights.fill(0.25)
        self.bias = bias
    
    def __call__(self, x: np.ndarray) -> np.ndarray:
        y = x @ self.weights.T
        if self.bias is not None:
            y += self.bias
        return y
    
def ReLU(x: np.ndarray) -> np.ndarray:
    return np.clip(x, min=0)

# Model

In [105]:
class Network():
    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int, bias: np.ndarray = None):
        self.layer1 = Linear(input_dim, hidden_dim, bias)
        self.layer2 = Linear(hidden_dim, output_dim, bias)
    
    def __call__(self, input: np.ndarray) -> np.ndarray:
        x = input.reshape(1, -1)
        x = ReLU(self.layer1(x))
        x = self.layer2(x)

        return x
    
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        self.linear_relu_stack.apply(init_weights)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

def init_weights(m):
    if isinstance(m, nn.Linear):
        m.weight.data.fill_(0.25)
        m.bias.data.fill_(0)


In [106]:
mynet = Network(28*28, 512, 10)
torchnet = NeuralNetwork()

train_features, train_labels = next(iter(train_dataloader))
print(torchnet(train_features[0]))
print(mynet(train_features[0].numpy()))

tensor([[9568.2510, 9568.2510, 9568.2510, 9568.2510, 9568.2510, 9568.2510,
         9568.2510, 9568.2510, 9568.2510, 9568.2510]],
       grad_fn=<AddmmBackward0>)
[[9568.251 9568.251 9568.251 9568.251 9568.251 9568.251 9568.251 9568.251
  9568.251 9568.251]]
